# Implementazione algoritmo
* 6/3/2021




## Cose da fare
1. Implementazione dell'algoritmo sviluppato dai medici
2. Induzione di un decision tree 
3. Giocare con gli hyperparamethers del decision tree indotto e riflettere sulle regole che esso utilizza per la classificazione

# Criteri inclusione pazienti nella popolazione studiata
1. Pazienti affetti da un sospettato tumore alla parotide che si sono poi sottoposti ad operazione
2. Pazienti di età non inferiore ai 18 anni
3. Pazienti che si sono già sottoposti o verranno sottoposti a MRI

# Criteri esclusione pazienti dalla popolazione studiata
1. Lesioni non-neoplastiche rilevate con esame istologico
2. Pazienti che hanno rifiutato operazioni chirurgiche
3. Lesioni molto piccole (<1 cm), lesioni cistiche (falsi positivi durante risonanza)


# Design dell'algoritmo

## Lesione parotidea

## Primo step: metodi convenzionali e MRI
Utilizzo di tecniche istologiche per determinare se sono presenti segni macroscopici di malignità (hypointensity, eterogeneità, infiltrazione in altre strutture). Se sono presenti segni di malignità macroscopici, ad esempio se c'è inflitrazione in strutture adiacenti, allora il tumore è classificato come maligno.

## Secondo step: diffusion weighted imaging
Misura dell'ADC. Se inferiore a 0.6 allora il tumore è classificato come linfoma. Se superiore 1.3 è classificato come benigno. Se è il valore dell'ADC è compreso tra questi due valori si prosegue con l'algoritmo

## Terzo step: perfusion studies
Analisi del tipo di curva TIC. Curva DCE (tempo che la lesione impiega per tornare nella condizione iniziale dopo l'im pulso). La curva può essere crescente (tipo A), decrescente (tipo B), pseudocostante (tipo C) o indefinita (tipo C).

# Output algoritmo
1. Malignant high-grade
2. Lymphoma
3. Pleomprphic adenoma
4. Warthin tumor
5. Benign tumor
6. Indeterminet lesion
7. Malignant tumor

# T2
Nel dataset 0 codifica un valore T2 alto, 1 codifica un valore basso. 

# Obbiettivo
Eliminare la classe indeterminato, considerare solo le prime 5 classi.

Per iniziare implemento il loro algoritmo. E vedo se l'output di questo algoritmo corrisponde all'output fornito


L'id del paziente non è feature da prendere in considerazione per la classificazione.
Da età a t2 sono feature, quello che segue è il loro output

Nell'implementazione del loro algoritmo tengo conto delle feature sopra indicate ma non dell'età.

Il codice istologico rappresenta la vera categoria del tumore, derivato dall'analisi delle sue cellule.

6 e 7 sono categorie indeterminate per algo (?)
1 è carcinoma (?)


# Assunzioni sugli alberi
Le etichette 1 e 6 sono da unire (?)
Tu lavori sull'albero di decisione e giochi sulla sua profondità massima
Ogni foglia deve avere almeno due pazienti
L'albero non deve essere troppo profondo--> se è convoluto significa che sta imparando a memoria, non impara relazione tra le features


cross validation-->dividi dati in 5 blocchi da 20-->calcola errore sul gruppo sui cui non alleni albero

sklearn.model_selection per calcolo errore

Puoi provare anche a fare 3 LOO. 3 samples messi da parte, alleni sul resto e calcoli errore sui 3 .

Una volta indotti gli alberi visualizzali per capire quali regole applicano per la classificazione.
Se hai multiple domande sulla stessa feature-->potrebbe essere overfitting

Per l'induzione degli alberi non usare gini, usa l'entropia

Tieni in considerazione max_depth --> quante regole puoi concatenare --> non troppo elevata per evitare overfitting --> se è troppo profondo apprende rumore

Ogni foglia deve avere almeno due campioni --> setta min_sample_leaf opportunamente

Ragiona su cosa l'albero ha appreso e confrontalo con l'algoritmo dato 

In [163]:
#Dataset come è stato fornito

import pandas as pd

df = pd.read_excel('Db_parotide_Def_REV_Samuel.xlsx')
df.head()

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,"2,6",D,0,0,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,"2,44",D,0,0,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
2,9,81,1,"0,7",C,1,1,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,21/03/2019,"Positivo per CTM, carcinoma squamocellulare",3,6,15/05/2019
3,1,28,0,"1,6",A,1,3,1,CARCINOMA,CARCINOMA DEI DOTTI SALIVARI,...,1,1,1,1,0,2016-06-10 00:00:00,sospetto per CTM,5,5,25/01/2017
4,7,51,0,"1,4",A,0,3,1,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,...,1,1,1,1,0,13/09/2018,Positivo per CTM. Carcinoma squamocellulare,3,6,2018-10-10 00:00:00


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ID                              107 non-null    int64 
 1   eta                             107 non-null    int64 
 2   Segni macro malignità           107 non-null    int64 
 3   ADC                             107 non-null    object
 4   TIC type                        107 non-null    object
 5   T2                              107 non-null    int64 
 6   Output Algoritmo                107 non-null    int64 
 7   COD ISTO                        107 non-null    int64 
 8   ISTO DEFINITIVO-PULITO          107 non-null    object
 9   ISTO DEFINITIVO                 107 non-null    object
 10  ISTO B/M                        107 non-null    int64 
 11  COD ISTO.1                      107 non-null    int64 
 12  CONCORD CODICE FNAC-ISTO B/M    107 non-null    ob

In [165]:
# Non sembrano esserci celle null nell'intero dataframe
# Ora procedo a ricavare il sotto-dataframe su cui devo lavorare per implementare l'algoritmo
#cambio il tipo dei valori ADC da str a float
#df è il dataframe di riferimento
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)


#estraggo dal dataset solo le feature di interesse per l'implementazione dell'algoritmo

dfAlgo = df.iloc[:,2:7]
dfAlgo.head()

,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo
0,0,2.60,D,0,0
1,0,2.44,D,0,0
2,1,0.70,C,1,1
3,0,1.60,A,1,3
4,0,1.40,A,0,3


In [166]:
#implemento l'algoritmo definito dai medici
#in questo caso non prendo in considerazione l'età dei pazienti, e ovviamente non cosidero l'output dell'algoritmo 
#stesso come feature da valutare
X = dfAlgo.iloc[:,:-1].copy()
X

,Segni macro malignità,ADC,TIC type,T2
0,0,2.60,D,0
1,0,2.44,D,0
2,1,0.70,C,1
3,0,1.60,A,1
4,0,1.40,A,0
...,...,...,...,...
102,0,1.30,C,1
103,0,1.90,A,0
104,0,2.10,A,0
105,0,1.80,A,0


In [167]:
y = dfAlgo['Output Algoritmo'].copy()
y

0      0
1      0
2      1
3      3
4      3
      ..
102    3
103    3
104    3
105    3
106    5
Name: Output Algoritmo, Length: 107, dtype: int64

In [168]:
#Recap classificazione
#1-->Malignant high grade
#2-->Lymphoma
#3-->Pleomorphic adenoma
#4-->Warthin tumor
#5-->benign tumor
#6-->indetermined
#7-->malignant


#Implementazione dell'algoritmo utilizzato dai medici

def algoritmo(input):#questa funzione prende come input una  una riga del dataset (X)
    
    #valutazione presenza segni macroscopici di malignità
    
    if input['Segni macro malignità'] == 1: #nodal involvment/infiltration of adjacent structures
        return 1  #malignant high-grade tumor
    
    #valutazione valori ADC
    
    if input['ADC'] <= 0.6:
        return 2  #lymphoma
    
    if input['ADC'] >= 1.3:
        return 3  #pleomorphic adenoma/cyst
    
    #sospetto di malignità
    
    if input['TIC type'] == 'A':
        if input['T2'] == 0:
            return 5 #benign tumor
        else:
            return 6 #indeterminate lesion
        
    if input['TIC type'] == 'B':
        return 4  #warthin tumor
    
    else:
        if input['T2'] == 0:
            return 6 #indeterminate lesion
        else:
            return 7 #malignant tumor

In [169]:
#devo applicare l'algoritmo a ciascuna riga del dataframe
#ottengo una serie dei risultati dell'applicazione dell'algoritmo implementato

results = X.apply(algoritmo, axis = 1)
results

0      3
1      3
2      1
3      3
4      3
      ..
102    3
103    3
104    3
105    3
106    5
Length: 107, dtype: int64

In [170]:
#ora confronto i risultati del mio algoritmo con quelli forniti e presenti nel dataframe  e calcolo il numero
#di risultati coincidenti
sum((results == y)*1)

104

In [171]:
#poichè le entry del df erano 107, solo in 3 casi vi è non corrispondenza
#Individuiamo questi pazienti

df.loc[results != y]

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,0,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,0,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
20,50,63,0,1.50,D,0,0,3,ADENOMA PLEOMORFO,ADENOMA PLEOMORFO,...,1,1,1,1,0,2019-04-06 00:00:00,adenoma pleomorfo,1,4a,21/10/2019


In [172]:
(results[0], results[1], results[20])

(3, 3, 3)

In [173]:
#tutti e tre sarebbero essere dovuti classificati come Pleomorphic adenoma
#inoltre il valore 0 non codifica nessuna classe
#modifico alcuni valori nell'ultima colonna del dataframe principale
df.iat[0,6] = 3
df.iat[1,6] = 3
df.iat[20,6] = 3
df.loc[results != y]

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,3,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,3,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
20,50,63,0,1.50,D,0,3,3,ADENOMA PLEOMORFO,ADENOMA PLEOMORFO,...,1,1,1,1,0,2019-04-06 00:00:00,adenoma pleomorfo,1,4a,21/10/2019


In [175]:
#Ora tento di indurre un albero di decisione a partire dal dataset fornito

#Uso COD ISTO come label da associare ai samples e dunque da predirre. Questi valori codificano il vero tipo 
#del tumore.

#dfAlbero sarà il dataframe di riferimento per il training del modello

y = df['COD ISTO'].copy()
X_not_encoded = df.copy().loc[:,'eta': 'T2']
X_not_encoded

,eta,Segni macro malignità,ADC,TIC type,T2
0,33,0,2.60,D,0
1,50,0,2.44,D,0
2,81,1,0.70,C,1
3,28,0,1.60,A,1
4,51,0,1.40,A,0
...,...,...,...,...,...
102,27,0,1.30,C,1
103,50,0,1.90,A,0
104,62,0,2.10,A,0
105,37,0,1.80,A,0


In [176]:
#Devo gestire il fatto che sklearn manipola solo variabili numeriche e non categoriche
X_not_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   eta                    107 non-null    int64  
 1   Segni macro malignità  107 non-null    int64  
 2   ADC                    107 non-null    float64
 3   TIC type               107 non-null    object 
 4   T2                     107 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 4.3+ KB


In [177]:
#applico conversione con one-hot-encoding get_dummies da pandas
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
X

,eta,Segni macro malignità,ADC,T2,TIC type_A,TIC type_B,TIC type_C,TIC type_D
0,33,0,2.60,0,0,0,0,1
1,50,0,2.44,0,0,0,0,1
2,81,1,0.70,1,0,0,1,0
3,28,0,1.60,1,1,0,0,0
4,51,0,1.40,0,1,0,0,0
...,...,...,...,...,...,...,...,...
102,27,0,1.30,1,0,0,1,0
103,50,0,1.90,0,1,0,0,0
104,62,0,2.10,0,1,0,0,0
105,37,0,1.80,0,1,0,0,0


In [189]:
y.value_counts()

3    54
4    27
1    16
2     5
5     5
Name: COD ISTO, dtype: int64

In [190]:
#Inizio ad indurre l'albero
#Faccio una cross validation stratificata a 5 fold e calcolo l'accuratezza media e la dev std
#5 fold così almeno la fold usata per il test contiene un rtappresentante della classe 5 e della classe 2
from sklearn import tree
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
import numpy as np
import graphviz


seed = 42
scoring = make_scorer(accuracy_score)

albero = tree.DecisionTreeClassifier(random_state = seed, criterion = 'entropy',
                                     max_depth = 4, min_samples_leaf = 2)

cv = StratifiedKFold(shuffle = True, n_splits = 5, random_state = None)

scores = cross_val_score(albero, X, y, cv = cv, scoring = scoring)
print(scores)
print('Acc: %0.3f\nStd dev: %0.3f' %(np.mean(scores), np.std(scores)))

[0.45454545 0.31818182 0.47619048 0.38095238 0.33333333]
Acc: 0.393
Std dev: 0.063


In [211]:
scoring = make_scorer(f1_score, average = 'weighted')#è macro che tiene conto di label imbalancement

scores = cross_val_score(albero, X, y, cv = cv, scoring = scoring)
print(scores)
print('Acc: %0.3f\nStd dev: %0.3f' %(np.mean(scores), np.std(scores)))

[0.26581028 0.37597403 0.32512315 0.49931973 0.40592741]
Acc: 0.374
Std dev: 0.079


In [212]:
#uso test size 0.2 stratificato così almeno il test set contiene un rappresentante della classe 2 e della classe 5
#provo altro approccio

for n in range(10):
    albero = tree.DecisionTreeClassifier(random_state = seed, criterion = 'entropy',
                                     max_depth = 4, min_samples_leaf = 2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y,
                                                        random_state = None, shuffle = True)
    
    albero.fit(X_train, y_train)


    y_pred = albero.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division = 0))
    print()
    

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         1
           3       0.47      0.64      0.54        11
           4       0.20      0.17      0.18         6
           5       0.00      0.00      0.00         1

    accuracy                           0.36        22
   macro avg       0.13      0.16      0.14        22
weighted avg       0.29      0.36      0.32        22


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         1
           3       0.55      0.55      0.55        11
           4       0.33      0.50      0.40         6
           5       0.00      0.00      0.00         1

    accuracy                           0.41        22
   macro avg       0.18      0.21      0.19        22
weighted avg       0.36      0.41      0.38        22


              precis

In [213]:
#ora uso grid search per trovare l'albero ottimo e lo stampo
from sklearn.model_selection import GridSearchCV

In [ ]:
tuned_parameters = {'criterion':['gini', 'entropy'],
                   'max_depth': list(range(2,11)),
                   'min_samples_leaf': list(range(2,11)),
                   'min_samples_split': list(range(y.value_counts().min(), 15)),
                   'class_weight':['balanced', None]}

cv = cv = StratifiedKFold(shuffle = True, n_splits = 5, random_state = None)
scoring = make_scorer(accuracy_score)
scores = []
for n in range(10):
    albero = tree.DecisionTreeClassifier(random_state = seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y,
                                                        random_state = None, shuffle = True)
    
    clf = GridSearchCV(albero, tuned_parameters, scoring = scoring, cv = cv, refit = True )
    clf.fit(X,y)
    print(clf.best_params_)
    print(clf.best_score_)
    scores.append(clf.best_score_)


print('Acc: %0.3f\nStd dev: %0.3f' %(np.mean(scores), np.std(scores)))

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 8, 'min_samples_split': 5}
0.4766233766233766
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 5}
0.4675324675324675
{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 6, 'min_samples_split': 14}
0.5038961038961038
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 5}
0.49567099567099565
